# DEMO for empathy classification

In this demo, we classify the empathy in text exchanges. 

We provide the most supported pattern for the classification of the exchange. 


# Set up

In [1]:


import pickle
import pandas as pd
import torch
import os
import sys
import random 
import re
#import classifier
from PBC4cip import PBC4cip
from PBC4cip.core.Evaluation import obtainAUCMulticlass
from PBC4cip.core.Helpers import get_col_dist, get_idx_val
from PBC4cip.core import Dataset as pbcDataset


#utilities for database management
import numpy as np
import pandas as pd
import os
import argparse

import train_classifier as trainer
import test_classifier as tester
import database_processing_package as data_processer

#relevant classifiers for annotating exchange feature
from classifiers.empathetic_intent import intent_prediction as ip
from classifiers.sentiment import sentiment_prediction as sp
from classifiers.epitome_mechanisms import epitome_predictor as epitome
from classifiers.nrc_vad_lexicon import lexicon_analysis as lexicon
from classifiers.course_grained_emotion import pretrained_32emotions as em32
from classifiers.course_grained_emotion import emotion_reductor as em_red
import database_processing_package as data_processer

from spellchecker import SpellChecker
from time import sleep


### Load main classification model

In [2]:
#Relevant directories
current_dir = os.getcwd() #get directory of the repository

#Database
database_dir = '/processed_databases/EmpatheticExchanges/EmpatheticExchanges.csv'
test_database_dir = '/processed_databases/EmpatheticExchanges/test.csv'
database = pd.read_csv(current_dir + database_dir)
test_db = pd.read_csv(current_dir + test_database_dir)

#Select an appropriate classification model in the Experiments folder
#number_of_model =90 #The number of the experiment for the model of interest

number_of_model = 104 #Best performing model without exchange number (broader pattern recognition)
model_directory = current_dir + '/Experiments/outputs/Experiment '+ str(number_of_model) + '/' + 'trained_pbc4cip.sav'
pbc = pickle.load(open(model_directory, 'rb'))

#Select features relevant for the model
att_lst = [attribute[0] for attribute in pbc.dataset.Attributes]
print('Features for the model: ')
for attribute in att_lst:
    print(attribute, end = ' ')


Features for the model: 
s_negative s_neutral s_positive l_negative l_neutral l_positive predictions_ER valence_speaker arousal_speaker dominance_speaker valence_listener arousal_listener dominance_listener s_word_len l_word_len agreeing acknowledging encouraging consoling sympathizing suggesting questioning wishing neutral mimicry 

### Load supplementary classification models

In [3]:
sentiment_flag = 0
ex_num_flag = 0
epitome_flag = 0
vad_flag = 0
intent_flag = 0
mimicry_flag = 0
length_flag = 1 #always on
emo_labels_flag = 0


#get the classifiers necessary for processing the response
if {'s_negative','s_neutral','s_positive','l_negative','l_neutral','l_positive'} & set(att_lst): 
    sentiment_flag = 1
    empIntSubDir = './classifiers/empathetic_intent/'
    sent_model, sent_tokenzr = sp.loadSentimentModel() #get model and tokenizer
if {'exchange_number'} & set(att_lst): 
    ex_num_flag = 1
if {'predictions_ER','predictions_IP','predictions_EX'} & set(att_lst): 
    epitome_flag = 1
    epitome_empathy_classifier = epitome.load_epitome_classifier('classifiers/epitome_mechanisms/trained_models')
if {'valence_speaker','arousal_speaker','dominance_speaker','valence_listener','arousal_listener','dominance_listener'} & set(att_lst): 
    vad_flag = 1
    lexicon_df, wnl, stp_wrds = lexicon.setup_lexicon('classifiers/nrc_vad_lexicon/BipolarScale/NRC-VAD-Lexicon.txt')
if {'agreeing','acknowledging','encouraging','consoling','sympathizing',
    'suggesting','questioning','wishing','neutral'} & set(att_lst): 
    intent_flag = 1
    empIntSubDir = './classifiers/empathetic_intent/'
    model_intent,tokenizer_intent,device = ip.loadModelTokenizerAndDevice(empIntSubDir) #get model and parameters
if {'mimicry'} & set(att_lst): 
    mimicry_flag = 1
    lexicon_df, wnl, stp_wrds = lexicon.setup_lexicon('classifiers/nrc_vad_lexicon/BipolarScale/NRC-VAD-Lexicon.txt')
if {'speaker_emotion','listener_emotion'} & set(att_lst):
    emo_labels_flag = 1
    emo32_model, emo32_tokenzr = em32.load32EmotionsModel()

flag_array = [sentiment_flag,ex_num_flag,epitome_flag,vad_flag,intent_flag,mimicry_flag,length_flag,emo_labels_flag]


### Function to classify exchange empathy

In [4]:

#Main processing function

def predict_exchange_empathy(classifier, flag_array,ex_num, att_lst, speaker_utterance, listener_utterance):
    
    #Turn an string exchange into a dataframe
    dummy_dic = {feature: [0] for feature in att_lst}
    exchange_df = pd.DataFrame.from_dict(dummy_dic)
    exchange_df['speaker_utterance'] = speaker_utterance
    exchange_df['listener_utterance'] = listener_utterance

    #Add the features to the exchange dataframe according to the ones used by model. We use the same functions as the experiments, which is why we do everything through dataframes
    if flag_array[0] == 1: 
        #get sentiment
        exchange_df['speaker_sentiment'] = exchange_df.apply(data_processer.get_sentiment_probabilities,axis = 1, args = (sent_model,sent_tokenzr,'speaker_utterance')) 
        exchange_df[['s_negative','s_neutral', 's_positive']] = pd.DataFrame(exchange_df.speaker_sentiment.tolist(),index = exchange_df.index)
        exchange_df['listener_sentiment'] = exchange_df.apply(data_processer.get_sentiment_probabilities,axis = 1, args = (sent_model,sent_tokenzr,'listener_utterance')) 
        exchange_df[['l_negative','l_neutral', 'l_positive']] = pd.DataFrame(exchange_df.listener_sentiment.tolist(),index = exchange_df.index)
        exchange_df = exchange_df.drop(columns=['speaker_sentiment','listener_sentiment'])
    if flag_array[1] == 1:
        #set exchange number to the one we provide
        exchange_df['exchange_number'] = ex_num
    if flag_array[2] == 1:
        #get epitome communication mechanisms
        exchange_df = epitome.classify_epitome_values(epitome_empathy_classifier, exchange_df)
    if flag_array[3] == 1:
        #get VAD vectors
        exchange_df['vad_speaker'] = exchange_df['speaker_utterance'].apply(lexicon.get_avg_vad, args = (lexicon_df,wnl,stp_wrds)) 
        exchange_df['vad_listener'] = exchange_df['listener_utterance'].apply(lexicon.get_avg_vad, args = (lexicon_df,wnl,stp_wrds)) 
        exchange_df[['valence_speaker','arousal_speaker','dominance_speaker']] = pd.DataFrame(exchange_df.vad_speaker.tolist(),index = exchange_df.index)
        exchange_df[['valence_listener','arousal_listener','dominance_listener']] = pd.DataFrame(exchange_df.vad_listener.tolist(),index = exchange_df.index)
        exchange_df = exchange_df.drop(columns = ['vad_speaker','vad_listener'])
    if flag_array[4] == 1:
        #get word lengths
        exchange_df['s_word_len'] = exchange_df['speaker_utterance'].apply(data_processer.get_word_len) 
        exchange_df['l_word_len'] = exchange_df['listener_utterance'].apply(data_processer.get_word_len) 
    if flag_array[5] == 1: 
        #get intent
        exchange_df['utterance'] = exchange_df['listener_utterance']
        exchange_df['is_response'] = 1
        exchange_df['empathetic_intent'] = exchange_df.apply(data_processer.get_emp_intent_probabilities, axis=1, args = (model_intent,tokenizer_intent,device,'listener_utterance'))
        exchange_df[data_processer.intent_labels] = pd.DataFrame(exchange_df.empathetic_intent.tolist(),index = exchange_df.index)
        exchange_df = exchange_df.drop(columns=['empathetic_intent','utterance','is_response'])
    if  flag_array[6] == 1:
        if(flag_array[3] == 1):
            #get the emotional similarity, if it is more than 0.7 set mimicry to 1
            exchange_df['emotional_similarity'] = exchange_df.apply(data_processer.get_cosine_similarity,axis = 1) 
            exchange_df['mimicry'] = exchange_df.apply(lambda x: 1 if x['emotional_similarity'] > 0.7 else 0, axis = 1)
            exchange_df = exchange_df.drop(columns = ['emotional_similarity'])
        else: 
            #get emotional mimicry by first computing VAD vectors, since these are not used by the classifier, they are subsequently deleted. 
            exchange_df['vad_speaker'] = exchange_df['speaker_utterance'].apply(lexicon.get_avg_vad, args = (lexicon_df,wnl,stp_wrds)) 
            exchange_df['vad_listener'] = exchange_df['listener_utterance'].apply(lexicon.get_avg_vad, args = (lexicon_df,wnl,stp_wrds)) 
            exchange_df[['valence_speaker','arousal_speaker','dominance_speaker']] = pd.DataFrame(exchange_df.vad_speaker.tolist(),index = exchange_df.index)
            exchange_df[['valence_listener','arousal_listener','dominance_listener']] = pd.DataFrame(exchange_df.vad_listener.tolist(),index = exchange_df.index)
            exchange_df = exchange_df.drop(columns = ['vad_speaker','vad_listener'])                
            exchange_df['emotional_similarity'] = exchange_df.apply(data_processer.get_cosine_similarity,axis = 1) 
            exchange_df['mimicry'] = exchange_df.apply(lambda x: 1 if x['emotional_similarity'] > 0.7 else 0, axis = 1)
            exchange_df = exchange_df.drop(columns =  ['valence_speaker','arousal_speaker','dominance_speaker','valence_listener','arousal_listener','dominance_listener','emotional_similarity'])
        exchange_df['mimicry'] = exchange_df['mimicry'].astype('category')
        exchange_df['mimicry'] = exchange_df['mimicry'].astype('string')
    if flag_array[7] == 1:
        #get emotion labels
        exchange_df['speaker_emotion'] = exchange_df.apply(data_processer.get_emotion_label,axis = 1, args = (emo32_model,emo32_tokenzr,'speaker_utterance')) 
        exchange_df['listener_emotion'] = exchange_df.apply(data_processer.get_emotion_label,axis = 1, args = (emo32_model,emo32_tokenzr,'listener_utterance')) 
        exchange_df = em_red.reduce_emotion_labels_to_8('speaker_emotion',exchange_df)
        exchange_df = em_red.reduce_emotion_labels_to_8('listener_emotion',exchange_df)

    exchange_df = exchange_df[att_lst] #Feature order must match! 
    #Predict empathy
    empathy_prediction = classifier.predict(exchange_df) 

    #output exchange dataframe and prediction
    return exchange_df, empathy_prediction[-1] + 1

### Function to recommend a response based on features of the exchange

In [142]:

def fix_mimicry(list):
    #This just changes the mimicry fature to have a 'binary' descriptor instead of a list to make it hashable
    if(('mimicry', ['0', '1']) in list):
        mimicry_index = list.index(('mimicry', ['0', '1']))
        list[mimicry_index] = ('mimicry', 'binary')
    return list

def is_greater_than(feature_name, pattern_items):
    #We use this function to get if the feature in a pattern implies an increase of that feature
    for item in pattern_items:
        if feature_name in str(item) and '>' in str(item):
            return True
    return False

def recursive_search(search_control, intersections,items,role):
    #print('recursive')
    
    #start the search
    if search_control == 0:
        if len(intersections[0]) > 0:
            return recursive_search(1,intersections,items,role)
        elif len(intersections[1]) > 0:
            return recursive_search(2,intersections,items,role)
        elif len(intersections[2]) > 0:
            return recursive_search(3,intersections,items,role)            
        elif len(intersections[3]) > 0:
            return recursive_search(4,intersections,items,role)
        elif len(intersections[4]) > 0:
            return recursive_search(5,intersections,items,role)
        elif len(intersections[5]) > 0:
            return recursive_search(6,intersections,items,role)
        else:
            #print('NO FEATURES AVAILABLE')
            return 'ERROR: No features available for this purpose'
    #The gist of this function is that it searches for features in the most representative pattern of a class of empathy. If it finds it and the pattern says that that feature is higher (>), we make a recommendation to increase that feature in the response
    if search_control == 1:
        #print('VAD')
        if ('valence_listener' in intersections[0]) and (is_greater_than('valence_listener', items)):
            #return 'Maybe we should be more positive!'
            return "Suggestion: Increase valence"
        elif 'arousal_listener' in intersections[0] and (is_greater_than('arousal_listener', items)):
            #return 'How about we say our feelings with more intensity!'
            #return "Suggestion: Increase arousal"
            return "Suggestion: Maybe we should focus on the feelings of the other person!" 
        elif 'arousal_listener' in intersections[0] and (is_greater_than('arousal_listener', items)):
            #return "Remember "+ role +" confidence is important!"
            return "Suggestion: "+ role +" should display more confidence (dominance)"
        else:
            #print('NO VAD')
            return recursive_search(search_control+1, intersections,items,role)
    elif search_control == 2:
        #print('Intent')
        if ('questioning' in intersections[1]) and (is_greater_than('questioning', items)):
            #return "Hey "+role+" why not ask them a question? I also want to know more"
            return "Suggestion: " + str(role) + ", I think you should ask them a question about what they say."
        elif ('acknowledging' in intersections[1]) and (is_greater_than('acknowledging', items)):
            #print(items)
            if ('s_positive <' in str(items)) or ('s_negative >' in str(items)) or ('valence_speaker <' in str(items)):
                return "Suggestion: Acknowledge their negative feelings, " + role
                #return "Oh no, Have you ever been in a similar situation "+ role+"?"
            else:
                return "Suggestion: Acknowledge their positive feelings, " + role
                #return "Oh nice, Have you ever been in a similar situation "+ role+"?"
        elif ('agreeing' in intersections[1]) and (is_greater_than('agreeing', items)):
            return "Suggestion: I think they were looking for agreement "+role+"."
            #return "I think they were looking for agreement "+role+"."
        elif ('encouraging' in intersections[1]) and (is_greater_than('encouraging', items)):
            return "Suggestion: My sensors say that we should encourage them about their feelings "+role+""
            #return "My sensors say that we should encourage them about their feelings "+role+"?"
        elif ('consoling' in intersections[1]) and (is_greater_than('consoling', items)):
            return "Suggestion: Consolation should be provided"
            #return "Oh no "+role+" Maybe we could say something to cheer them up!"       
        elif ('suggesting' in intersections[1]) and (is_greater_than('suggesting', items)):
            return "Suggestion: Suggest something they could do"
            #return 'What do you think they should do?'
        elif ('wishing' in intersections[1]) and (is_greater_than('wishing', items)):
            return "Suggestion: Humans wish eachother good things in these situations"
            #return "I hear humans wish eachother good things in these situations "+role+". Why don't you give it a try?"
        elif ('sympathizing' in intersections[1]) and (is_greater_than('sympathizing', items)):
            return "Suggestion: Respond with sympathy"
            #return 'Oh, I feel bad for them. Do you feel like that too?'  
        else:
            return recursive_search(search_control+1, intersections,items,role)
    elif search_control == 3:
        #print('Sentiment')
        if ('l_positive' in intersections[1]) and (is_greater_than('l_positive', items)):
            return "Suggestion: Be more positive"
            #return 'Hmm, maybe we should be more positive'
        elif ('l_negative' in intersections[2]) and (is_greater_than('l_negative', items)):
            #return "Oh, that's bad. Don't you agree "+role+"?"
            return "Suggestion: Acknowledge their negative feelings"
        else:
            return recursive_search(search_control+1, intersections,items,role)
    elif search_control == 4:
        #print('epitome')
        if ('predictions_ER' in intersections[3]) and (is_greater_than('predictions_ER', items)):
            return 'How about we say our feelings with more intensity!'
        elif ('predictions_EX' in intersections[3]) and (is_greater_than('predictions_EX', items)):
            return "Hey "+role+" why not ask them a question? I also want to know more"
        elif ('predictions_IP' in intersections[3]) and (is_greater_than('predictions_IP', items)):
            return "Do you understand what they mean?"   
        else:
            return recursive_search(search_control+1, intersections,items,role)
    elif search_control == 5:
        if len(intersections[4]) > 0:
            return 'Do you feel the same way?'
        else:
            return recursive_search(search_control+1, intersections,items,role)
    elif search_control == 6:
        if len(intersections[5]) > 0:
            return 'Maybe you should elaborate more'
        else:
            return recursive_search(search_control+1, intersections,items,role)
    else:
        #print('NO FEATURES AVAILABLE')
        return 'ERROR'


def compute_feature_intersections(feature_names):
    #Features from the 'listener' role
    intent_features = ['agreeing', 'acknowledging', 'encouraging', 'consoling', 'sympathizing', 'suggesting', 'questioning', 'wishing']
    sentiment_features = ['l_negative', 'l_neutral', 'l_positive']
    epitome_features = ['predictions_ER', 'predictions_EX', 'predictions_IP']
    length_features = ['l_word_len']
    vad_features = ['valence_listener', 'arousal_listener', 'dominance_listener']
    mimicry_features = ['mimicry']

    intent_intersection = set(intent_features).intersection(set(feature_names))
    vad_intersection = set(vad_features).intersection(set(feature_names))
    sentiment_intersection = set(sentiment_features).intersection(set(feature_names))
    epitome_intersection = set(sentiment_features).intersection(set(feature_names))
    mimicry_intersection = set(mimicry_features).intersection(set(feature_names))
    len_intersection = set(length_features).intersection(set(feature_names))

    intersection_arr = [vad_intersection,intent_intersection,sentiment_intersection,epitome_intersection,mimicry_intersection,len_intersection]
    return intersection_arr
    


def get_recommentation(classifier, exchange_df,role):
    #get most influential patterns
    
    emerging_patterns = classifier.EmergingPatterns #access the patterns mined by the classifier
    pattern_list = [] #patterns that cover the exchange
    for instance in exchange_df.to_numpy(): 
        for pattern in emerging_patterns:
            if pattern.IsMatch(instance):
                pattern_list.append(pattern)   
    count_lst = [pattern.Counts for pattern in pattern_list]
    count_arr = np.array(count_lst)
    most_support_low = pattern_list[count_arr[:,0].argmax()] #Pattern that most supports the lowest class
    most_support_mid = pattern_list[count_arr[:,1].argmax()] #Pattern that most supports the middle class
    most_support_high = pattern_list[count_arr[:,2].argmax()] #Pattern that most supports the high class

    #print(most_support_high)
    
    most_support_high_items = most_support_high.Items
    most_support_mid_items = most_support_mid.Items
    most_support_low_items = most_support_low.Items
    
    #get the features from most supported patterns
    most_support_high_features = [items.Feature for items in most_support_high.Items]
    most_support_mid_features = [items.Feature for items in most_support_mid.Items]
    most_support_low_features =[items.Feature for items in most_support_low.Items]

    most_support_high_features = fix_mimicry(most_support_high_features)
    most_support_mid_features = fix_mimicry(most_support_mid_features)
    most_support_low_features = fix_mimicry(most_support_low_features)


    #Obtain the features that differenciate the low features from the others
    features_only_on_low = set(most_support_low_features) - set(most_support_high_features) - set(most_support_mid_features) 
    features_only_on_mid = set(most_support_mid_features) - set(most_support_low_features)
    features_only_on_high = set(most_support_high_features) - set(most_support_low_features)

    #print(most_support_high)
    #print(features_only_on_high)
    
    #get the names of the features on the class of interest
    feature_names_high = [features[0] for features in features_only_on_high]
    feature_names_mid = [features[0] for features in features_only_on_mid]
    fearure_names_low = [features[0] for features in features_only_on_low]

    #First, we try to improve empathy using the features from the pattern most representative of the high empathy class
    intersections = compute_feature_intersections(feature_names_high)
    response = recursive_search(0,intersections,most_support_high_items, str(role))
    #print(type(response))
    if 'ERROR' in response:
        #if that fails, we try to get a response using the features from the middle empathy class
        intersections = compute_feature_intersections(feature_names_mid)
        response = recursive_search(0,intersections,most_support_mid_items,str(role))
        if 'ERROR' in response:
            #If that fails, we just give up (maybe we should try to do the reverse using the features on the low empathy class i.e. check if there are features we could reduce (<))
            print('Maybe we should talk about something else')
    
    #print(response)
    return response

### Function to judge an exchange and get a recommendation

In [6]:
def judge_exchange(classifier,flag_array, att_lst,speaker_utterance,listener_utterance,role):
    processed_exchange, pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst,speaker_utterance, listener_utterance)
    print(f"Empathy prediction for {role}: {pred}/3")
    if pred < 2: 
        print('Low empathy detected!')
        recommendation = get_recommentation(pbc, processed_exchange,role)
        print(f"interjection by Haru: {recommendation}")
        #print(recommendation)
        return False
    return True

def get_input():
    bad_input_flag = True
    while bad_input_flag:
        utterance = input("Provide input: ")
        if utterance.lower() == '':
            print('Please provide valid input')
        else:
            bad_input_flag = False    
    return utterance 

In [7]:
judge_exchange(pbc,flag_array,att_lst,'I hate when my wife and son are away from me', "Oh ok. ","listener")
judge_exchange(pbc,flag_array,att_lst,'I hate when my wife and son are away from me', "I've never been in that situation but that is understandable. ","listener")


Empathy prediction for listener: 1/3
Low empathy detected!
interjection by Haru: Suggestion: Acknowledge their negative feelings, listener


Empathy prediction for listener: 2/3


True

# Inference

## Examples of inference and recommendations

In [8]:

judge_exchange(pbc,flag_array,att_lst,'finally got my house, I do not have to deal with apartment living anymore', "apartments are ok","listener")

judge_exchange(pbc,flag_array,att_lst,'finally got my house, I do not have to deal with apartment living anymore', "apartments are ok, you shouldn't knock them","listener")

judge_exchange(pbc,flag_array,att_lst,'finally got my house, I do not have to deal with apartment living anymore', "That's great I love living in my apartment but I'm happy for you","listener")



Empathy prediction for listener: 1/3
Low empathy detected!
interjection by Haru: Suggestion: Suggest something they could do


Empathy prediction for listener: 3/3


Empathy prediction for listener: 3/3


True

In [9]:

judge_exchange(pbc,flag_array,att_lst,'I cannot wait for the newest Pokemon game, it looks amazing to me!', "pokemon is just ok","listener")


judge_exchange(pbc,flag_array,att_lst,'I cannot wait for the newest Pokemon game, it looks amazing to me!', "Oh well it's not bad. I had fun with pokemon when I was 10","listener")


Empathy prediction for listener: 1/3
Low empathy detected!
interjection by Haru: Suggestion: Acknowledge their positive feelings, listener


Empathy prediction for listener: 2/3


True

## Automatic conversation loop

In [10]:
spkr = ["I cannot wait for the newest Pokemon game, it looks amazing to me!", "I see your point, but I still think it is fun",'abortsequence']
lstnr = ['pokemon is just ok',"Oh well it's not bad. I had fun with pokemon when I was 10", "I relate to that, I listen to old songs from my childhood",'abortsequence']

speaker_utterances = []
listener_utterances = []

conversation_end = False
#For a short conversation 
for i in range(2):    
    if i>0:
        j = i
        while not good_speaker:
            prompt = spkr[j]
            print(f"Speaker turn: {prompt}")
            print('Speaker_empathy')
            good_speaker = judge_exchange(pbc, flag_array, att_lst, listener_utterances[i-1],prompt,'speaker')
            j = j+1
            #judgement_on_speaker = False
            #print(f'judgement on speaker {judgement_on_speaker}')
    else:      
        prompt = spkr[i]
        print(f"Speaker turn: {prompt}")
        if 'abortsequence' in prompt:
            break
        good_speaker = False    
        
    
    
    speaker_utterances.append(prompt) #We append the successful utterance to the record
    good_listener = False
    #Keep listener hostage while they do not provide empathetic responses
    j = i
    while not good_listener:
        answer = lstnr[j]
        print(f"Listener turn: {answer}")
        if 'abortsequence' in answer:
            break
        good_listener = judge_exchange(pbc, flag_array, att_lst, speaker_utterances[i], answer,'listener')
        j = j+1
        #print(f"Speaker: {speaker_utterances[i]}")
        #print(f"Listener: {listener_utterances[i]}")     
    listener_utterances.append(answer)


print(speaker_utterances)
print(listener_utterances)


Speaker turn: I cannot wait for the newest Pokemon game, it looks amazing to me!
Listener turn: pokemon is just ok


Empathy prediction for listener: 1/3
Low empathy detected!
interjection by Haru: Suggestion: Acknowledge their positive feelings, listener
Listener turn: Oh well it's not bad. I had fun with pokemon when I was 10


Empathy prediction for listener: 2/3
Speaker turn: I see your point, but I still think it is fun
Speaker_empathy


Empathy prediction for speaker: 3/3
Listener turn: Oh well it's not bad. I had fun with pokemon when I was 10


Empathy prediction for listener: 2/3
['I cannot wait for the newest Pokemon game, it looks amazing to me!', 'I see your point, but I still think it is fun']
["Oh well it's not bad. I had fun with pokemon when I was 10", "Oh well it's not bad. I had fun with pokemon when I was 10"]


In [11]:
spkr = ["I cannot wait for the newest Pokemon game, it looks amazing to me!", "I see your point, but I still think it is fun",'abortsequence']
lstnr = ['pokemon is just ok',"Oh well it's not bad. I had fun with pokemon when I was 10", "Yeah it was to me, maybe I should try the new one",'abortsequence']

speaker_utterances = []
listener_utterances = []

conversation_end = False
#For a short conversation

j = 0
i = 0

while not conversation_end:    
    if j>0:
        good_speaker = False    
        while not good_speaker:
            prompt = spkr[j]
            print(f"Speaker turn: {prompt}")
            print('Speaker_empathy')
            good_speaker = judge_exchange(pbc, flag_array, att_lst, lstnr[i-1],prompt,'speaker')
            j = j+1
            if j + 1 >= len(spkr):
                conversation_end = True
            #judgement_on_speaker = False
            #print(f'judgement on speaker {judgement_on_speaker}')
    else:      
        prompt = spkr[i]
        print(f"Speaker turn: {prompt}")
        if 'abortsequence' in prompt:
            break
        good_speaker = False    
        if j + 1 >= len(spkr):
            conversation_end = True
        else:
            j += 1        
    speaker_utterances.append(prompt) #We append the successful utterance to the record
    good_listener = False
    #Keep listener hostage while they do not provide empathetic responses
    while not good_listener:
        answer = lstnr[i]
        print(f"Listener turn: {answer}")
        if 'abortsequence' in answer:
            conversation_end = True
            break
        good_listener = judge_exchange(pbc, flag_array, att_lst, spkr[j], answer,'listener')
        i = i+1
        #print(f"Speaker: {speaker_utterances[i]}")
        #print(f"Listener: {listener_utterances[i]}")     
    listener_utterances.append(answer)
    if i + 1 >= len(lstnr):
        conversation_end = True
    else:
        i += 1
    print(f'{i} {j}')
        


print(speaker_utterances)
print(listener_utterances)

Speaker turn: I cannot wait for the newest Pokemon game, it looks amazing to me!
Listener turn: pokemon is just ok


Empathy prediction for listener: 1/3
Low empathy detected!
interjection by Haru: Suggestion: Acknowledge their positive feelings, listener
Listener turn: Oh well it's not bad. I had fun with pokemon when I was 10


Empathy prediction for listener: 2/3
3 1
Speaker turn: I see your point, but I still think it is fun
Speaker_empathy


Empathy prediction for speaker: 3/3
Listener turn: abortsequence
3 2
['I cannot wait for the newest Pokemon game, it looks amazing to me!', 'I see your point, but I still think it is fun']
["Oh well it's not bad. I had fun with pokemon when I was 10", 'abortsequence']


## Conversation Loop

In [12]:
speaker_utterances = []
listener_utterances = []
#For a short conversation 
for i in range(2):    
    print("Speaker, ", end = '')
    prompt = get_input()
    if 'abortsequence' in prompt:
        break
    speaker_utterances.append(prompt) 
    good_exchange = False
    #Keep listener hostage while they do not provide empathetic responses
    while not good_exchange:
        print("Listener,  ", end = '')
        answer = get_input()
        if 'abortsequence' in answer:
            break
        listener_utterances.append(answer)
        print(f"Speaker: {speaker_utterances[i]}")
        print(f"Listener: {listener_utterances[i]}")
        good_exchange = judge_exchange(pbc, flag_array, att_lst, speaker_utterances[i], listener_utterances[i],'listener')
    if i>0:
        judgement_on_speaker = judge_exchange(pbc, flag_array, att_lst, listener_utterances[i-1],speaker_utterances[i],'speaker')
        #judgement_on_speaker = False
        print(f'judgement on speaker {judgement_on_speaker}')

print(speaker_utterances)
print(listener_utterances)


Speaker, 

KeyboardInterrupt: Interrupted by user

In [13]:
def get_input():
    bad_input_flag = True
    while bad_input_flag:
        utterance = input("Provide input: ")
        if utterance.lower() == '':
            print('Please provide valid input')
        else:
            bad_input_flag = False    
    return utterance 

utterance = get_input()

print(utterance)

KeyboardInterrupt: Interrupted by user

In [14]:
processed_exchange, y_pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, 'How are you?', "Doing pretty good, how about yourself?")
processed_exchange, y_pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, "I'm doing ok, just had a dental implant done", "Oh, ouch, that must have been pretty painful")
processed_exchange, y_pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, "Yeah, It was. But not as bad as you would think", "Great to hear, and now you have a shiny new tooth!")
processed_exchange, y_pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, "Yes! That makes it all worth it in the end", "For sure, despite the bad you get something great out of it")

In [15]:
judge_exchange(pbc, flag_array, att_lst, "Oh, ouch, that must have been pretty painful",  "Yeah, It was. But not as bad as you would think")

TypeError: judge_exchange() missing 1 required positional argument: 'role'

In [16]:
candidates = []
for i in database['prompt']:
    if(len(database[database['prompt'] == i]) >= 4 ):
        #print(database[database['prompt'] == i])
        candidates.append(i)
set(candidates)

{'A co-worker of mine got a promotion over me.',
 'I almost stepped on a snake today.',
 'I applied for a job last week',
 'I ate a whole pack of cookies',
 "I can't wait for this halloween. I have the cutest theme for my kids",
 'I cannot wait for the new super smash bros for the nintendo switch',
 'I enjoy working on and driving my old truck.',
 "I forgot to get my son a bag of chips from the dollar store when I went out today - especially since I told him I'd pick up some on my way home.",
 'I found out that I was one of the top performers in the Mid-South district for my job! I knew I could do it!',
 'I found out that my wife is pregnant today.',
 'I have done lots of research for an interview this week',
 'I just witnessed someone steal from an old woman walking across the street. Boils my blood!',
 "I lost a job I had for many years. I wasn't sure what I would do_comma_ but I felt things would work out",
 'I ordered McDonalds today on UberEats. It was terrible.',
 "I received an 

In [18]:
processed_exchange, pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, 'Super sad today. It is the weekend pretty much and I have a hard time with loneliness on the weekends especially.', "I am sorry. Maybe not focus as friday-sunday as the weekend_comma_ but as any other day. And do things you would normally do on other days.")
print(f"Predicted empathy: {pred}")
if pred < 2: 
    print('Low empathy detected')
    print('response formulated by Haru')
    recommendation = get_recommentation(pbc, processed_exchange, 'listener')
    print(recommendation)

processed_exchange, pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, 'Super sad today. It is the weekend and I have a hard time with loneliness on the weekends especially.', "I love the weekends actually")
print(f"Predicted empathy: {pred}")
if pred < 2: 
    print('Low empathy detected')
    print('response formulated by Haru')
    recommendation = get_recommentation(pbc, processed_exchange, 'listener')
    print(recommendation)


processed_exchange, pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, 'Super sad today. It is the weekend and I have a hard time with loneliness on the weekends especially.', "Sorry to hear you are feeling lonely. But don't worry I am your friend")
print(f"Predicted empathy: {pred}")
if pred < 2: 
    print('Low empathy detected')
    print('response formulated by Haru')
    recommendation = get_recommentation(pbc, processed_exchange, 'listener')
    print(recommendation)

processed_exchange, pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, 'Super sad today. It is the weekend pretty much and I have a hard time with loneliness on the weekends especially.', "Sorry to hear that, maybe you should go join a club to meet new people")
print(f"Predicted empathy: {pred}")
if pred < 2: 
    print('Low empathy detected')
    print('response formulated by Haru')
    recommendation = get_recommentation(pbc, processed_exchange, 'listener')
    print(recommendation)


Predicted empathy: 3


Predicted empathy: 1
Low empathy detected
response formulated by Haru
Suggestion: Acknowledge their negative feelings


Predicted empathy: 3


Predicted empathy: 3


In [149]:

print("First exchange")
judge_exchange(pbc,flag_array,att_lst,"i loved taking care of my sisters pet", "Huh, is that so", 'listener')
judge_exchange(pbc,flag_array,att_lst,"i loved taking care of my sisters pet", "It's cool that you loved that", 'listener')
print("Second exchange")
judge_exchange(pbc,flag_array,att_lst,"Yeah! I have loved animals since then especially dogs", "Dogs are very cute. Cats too", "listener")
print("Third exchange")
judge_exchange(pbc,flag_array,att_lst,"Oh my gosh yes. Cats are so fluffy and cuddly", "They are! I love petting them", "listener")
print("Fourth exchange")
judge_exchange(pbc,flag_array,att_lst,"You know i really enjoy having pets they bring a new life into an empty feeling house", "Yes I only have one cat. How about you?", "listener")
print("Fifth exchange")
judge_exchange(pbc,flag_array,att_lst,"we have a cat, a dog, a bunny, and a betta fish!", "Those are many pets", "listener")
judge_exchange(pbc,flag_array,att_lst,"we have a cat, a dog, a bunny, and a betta fish!", "Those are many pets, how do you manage?", "listener")
print("Six exchange")
judge_exchange(pbc,flag_array,att_lst,"It is a lot of work. But their little faces are so worth it", "Yes they are I bet you feel proud", "listener")


First exchange


Empathy prediction for listener: 1/3
Low empathy detected!
interjection by Haru: Suggestion: My sensors say that we should encourage them about their feelings listener


Empathy prediction for listener: 3/3
Second exchange


Empathy prediction for listener: 3/3
Third exchange


Empathy prediction for listener: 2/3
Fourth exchange


Empathy prediction for listener: 3/3
Fifth exchange


Empathy prediction for listener: 1/3
Low empathy detected!
interjection by Haru: Suggestion: listener, I think you should ask them a question about what they say.


Empathy prediction for listener: 2/3
Six exchange


Empathy prediction for listener: 3/3


True

In [45]:
processed_exchange, y_pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, 'At the time there was a friend that told me that i could not jump over him_comma_ then i jumped over him.', "Neat")
processed_exchange, y_pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, "I'm doing ok, just had a dental implant done", "Oh, ouch, that must have been pretty painful")
processed_exchange, y_pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, "Yeah, It was. But not as bad as you would think", "Great to hear, and now you have a shiny new tooth!")
processed_exchange, y_pred  = predict_exchange_empathy(pbc, flag_array, 1, att_lst, "Yes! That makes it all worth it in the end", "For sure, despite the bad you get something great out of it")